# Exemple Python Pandas et Singlestore

Variables d'environnement

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from a .env file
load_dotenv()

# Access the variables
HOST_DML = os.getenv("HOST_DML")
USERDB = os.getenv("USERDB")
PASSDB = os.getenv("PASSDB")

Connection à la base de donnée source

In [2]:
import singlestoredb as s2
source = s2.connect(host=HOST_DML, port='3306', user=USERDB,
                  password=PASSDB, database='tpch')

Requête SQL

In [3]:
import pandas as pd

query_sql = """
    SELECT YEAR(o_orderdate) AS order_year,
           COUNT(*) AS order_count,
           SUM(o_totalprice) AS total_spent
    FROM orders
    WHERE o_orderdate >= '1995-01-01'
    GROUP BY YEAR(o_orderdate);
"""

with source.cursor() as cur:
    cur.execute(query_sql)
    source.commit()

# Fetch the results into a DataFrame
df = pd.read_sql(query_sql, source)

/tmp/ipykernel_14909/2202941423.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_sql, source)


In [4]:
# Display the first few rows of the DataFrame
df.head()

,order_year,order_count,total_spent
0,1998,13341863,2.016259e+12
1,1995,22754973,3.438388e+12
2,1996,22819651,3.449188e+12
3,1997,22760815,3.439552e+12


Connection à la base de donnée Datamart

In [5]:
import sqlalchemy as sa
from sqlalchemy import create_engine

engine_dtm = create_engine(f'singlestoredb://{USERDB}:{PASSDB}@{HOST_DML}/dtm')
target = engine_dtm.connect()

# Write the DataFrame to a new table in the target database
df.to_sql('orders_summary_python', con=target, index=False, if_exists='replace')

4

Exemple select avec la syntaxe SQLAlchemy

In [6]:
# Create a metadata object for the database
if sa.__version__.startswith('1'):
    db = sa.MetaData(bind=engine_dtm)
    sa.MetaData.reflect(db)
else:
    db = sa.MetaData()
    db.reflect(bind=engine_dtm)

# Get the iris table from reflected data
sa_tbl = db.tables['orders_summary_python']

# Query the iris table
query = sa.select(sa_tbl).limit(10)

# Print results
for row in target.execute(query):
    print(row)

(1997, 22760815, 3439550000000.0)
(1995, 22754973, 3438390000000.0)
(1998, 13341863, 2016260000000.0)
(1996, 22819651, 3449190000000.0)


Jointure entre deux tables dans deux bases de données différentes

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text

# Create engine
engine = create_engine(f'singlestoredb://{USERDB}:{PASSDB}@{HOST_DML}')

# Define SQL query
query = """
    SELECT 
        o.o_orderkey,
        o.o_orderdate,
        o.o_custkey,
        c.c_custkey,
        c.c_name
    FROM dtm.customer_scb AS c
    LEFT JOIN tpch.orders AS o
      ON o.o_custkey = c.c_custkey
    LIMIT 10
"""

with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS dtm.customer_orders_join"))

# Use connection to read SQL into DataFrame
with engine.connect() as conn:
    df = pd.read_sql(text(query), conn)

# Display the DataFrame
print(df.head())

   o_orderkey o_orderdate  o_custkey  c_custkey              c_name
0   105200582  1996-08-19    5975891    5975891  Customer#005975891
1   105284615  1994-01-24   10812580   10812580  Customer#010812580
2   105511777  1995-06-23   14426798   14426798  Customer#014426798
3   105552583  1994-03-12   11870152   11870152  Customer#011870152
4   105582881  1994-04-24    9999514    9999514  Customer#009999514
